<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat

`flaml.autogen` offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framwork allows tool use and human participance through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/FLAML/docs/Use-Cases/Autogen#agents).

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
%%capture --no-stderr
# %pip install flaml[autogen]~=2.0.1

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
from flaml import autogen

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt-4-32k-0314", "gpt-4-32k-v0314"],
        # "model": ["gpt-4", "gpt-4-0314"],
        # "model": ["gpt-4", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-4-32k"],
        "api_base": "https://flaml-east.openai.azure.com/",
    },
)
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
            "gpt",
        },
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [3]:
gpt4_config = {
    "seed": 48,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "request_timeout": 120,
}
gpt3_config = {
    "seed": 48,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt35,
}
human = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
#    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.

Available functions you can import from utils:

def find_papers_pmc(query: str, size: int = 10):
    """
    Find papers in PubMed Central (PMC) using Europe PMC API.
    Print the title, url, publication date, and abstract of each paper found.
    This function doesn't return anything.

    Args:
        query (str): The search query to use.
        size (int): The number of results to return (default 10).
    """

def search_arxiv(query, max_results=10):
    """
    Searches arXiv for the given query and returns the search results.

    Args:
        query (str): The search query.
        max_results (int, optional): The maximum number of search results to return. Defaults to 10.

    Returns:
        jresults (list): A list of dictionaries. Each dictionary contains fields such as 'title', 'authors', 'summary', and 'pdf_url'

    Example:
        >>> results = search_arxiv("attention is all you need")
        >>> print(results)
    """ 
''',
)
# coder.update_system_message(coder.system_message + '''
# ''')
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts without coding."""
)
planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
Available functions the engineer can import from utils:

def find_papers_pmc(query: str, size: int = 10):
    """
    Find papers in PubMed Central (PMC) using Europe PMC API.
    Print the title, url, publication date, and abstract of each paper found.
    This function doesn't return anything.

    Args:
        query (str): The search query to use.
        size (int): The number of results to return (default 10).
    """

def search_arxiv(query, max_results=10):
    """
    Searches arXiv for the given query and returns the search results.

    Args:
        query (str): The search query.
        max_results (int, optional): The maximum number of search results to return. Defaults to 10.

    Returns:
        jresults (list): A list of dictionaries. Each dictionary contains fields such as 'title', 'authors', 'summary', and 'pdf_url'

    Example:
        >>> results = search_arxiv("attention is all you need")
        >>> print(results)
    """ 
''',
    llm_config=gpt4_config,
)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "med"},
)
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. Check whether the code uses given functions correctly."
    '''
Available functions the engineer can import from utils:

def find_papers_pmc(query: str, size: int = 10):
    """
    Find papers in PubMed Central (PMC) using Europe PMC API.
    Print the title, url, publication date, and abstract of each paper found.
    This function doesn't return anything.

    Args:
        query (str): The search query to use.
        size (int): The number of results to return (default 10).
    """

def search_arxiv(query, max_results=10):
    """
    Searches arXiv for the given query and returns the search results.

    Args:
        query (str): The search query.
        max_results (int, optional): The maximum number of search results to return. Defaults to 10.

    Returns:
        jresults (list): A list of dictionaries. Each dictionary contains fields such as 'title', 'authors', 'summary', and 'pdf_url'

    Example:
        >>> results = search_arxiv("attention is all you need")
        >>> print(results)
    """
''',
    llm_config=gpt4_config,
)
groupchat = autogen.GroupChat(agents=[human, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Start Chat

In [4]:
human.initiate_chat(
    manager,
    message="""
find 30 latest papers on usage of generative AI in medicine and create a markdown table of different domains where it is being used.
""",
)

Admin (to chat_manager):


find 30 latest papers on usage of generative AI in medicine and create a markdown table of different domains where it is being used.


--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan:

1. The engineer will use the `find_papers_pmc` function to find the 30 latest papers on the usage of generative AI in medicine. The query will be "generative AI in medicine". The size will be set to 30.

2. The scientist will review the papers found by the engineer and identify the different domains where generative AI is being used in medicine. 

3. The scientist will then create a markdown table listing these domains. The table will have two columns: "Domain" and "Number of Papers". The "Domain" column will list the different domains identified, and the "Number of Papers" column will list the number of papers found for each domain.

4. The engineer will write a script to automate the creation of the markdown ta